In [ ]:
# default_exp core

# What_if_tool

In [ ]:
!pip install witwidget

#### Install ndoejs via conda 

In [ ]:
!conda install nodejs --yes

Solving environment: done

# All requested packages already installed.



#### Extensions for widgets 

In [ ]:
!jupyter nbextension install --py --symlink --sys-prefix witwidget
!jupyter nbextension enable --py --sys-prefix witwidget

2021-01-04 04:48:30.158764: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-01-04 04:48:30.158802: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Installing /opt/conda/lib/python3.7/site-packages/witwidget/static -> wit-widget
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable witwidget --py --sys-prefix
    
2021-01-04 04:48:35.959057: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-01-04 04:48:35.959097: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if 

In [ ]:
!jupyter labextension install wit-widget
!jupyter labextension install jupyter-widgets/jupyterlab-manager --yes

Building jupyterlab assets (build:prod:minimize)
Install labextension(s)

 Usage

    jupyter labextension install [--pin-version-as <alias,...>] <package...>

This installs JupyterLab extensions similar to yarn add or npm install.

Pass a list of comma seperate names to the --pin-version-as flag to use as
alises for the packages providers. This is useful to install multiple versions
of the same extension. These can be uninstalled with the alias you provided to
the flag, similar to the "alias" feature of yarn add.

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--no-build
    Defer building the app after the action.
--clean
    Cleanup intermediate files a

### Imports

In [ ]:
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

## Already Build Model

In [ ]:
PROJECT_ID = 'YOUR_PROJECT_ID'
MODEL_NAME = 'YOUR_MODEL_NAME'
VERSION_NAME = 'YOUR_VERSION_NAME'
TARGET_FEATURE = 'mortgage_status'
LABEL_VOCAB = ['denied', 'approved']

config_builder = (WitConfigBuilder(test_examples.tolist(), features.columns.tolist() + ['mortgage_status'])
  .set_ai_platform_model(PROJECT_ID, MODEL_NAME, VERSION_NAME, adjust_prediction=adjust_prediction)
  .set_target_feature(TARGET_FEATURE)
  .set_label_vocab(LABEL_VOCAB))


In [ ]:
WitWidget(config_builder, height=800)

### Define Helper function

In [ ]:
#@title Define helper functions {display-mode: "form"}

import pandas as pd
import numpy as np
import tensorflow as tf
import functools

# Creates a tf feature spec from the dataframe and columns specified.
def create_feature_spec(df, columns=None):
    feature_spec = {}
    if columns == None:
        columns = df.columns.values.tolist()
    for f in columns:
        if df[f].dtype is np.dtype(np.int64) or df[f].dtype is np.dtype(np.int32):
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
        elif df[f].dtype is np.dtype(np.float64):
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.float32)
        else:
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.string)
    return feature_spec

# Creates simple numeric and categorical feature columns from a feature spec and a
# list of columns from that spec to use.
#
# NOTE: Models might perform better with some feature engineering such as bucketed
# numeric columns and hash-bucket/embedding columns for categorical features.
def create_feature_columns(columns, feature_spec):
    ret = []
    for col in columns:
        if feature_spec[col].dtype is tf.int64 or feature_spec[col].dtype is tf.float32:
            ret.append(tf.feature_column.numeric_column(col))
        else:
            ret.append(tf.feature_column.indicator_column(
                tf.feature_column.categorical_column_with_vocabulary_list(col, list(df[col].unique()))))
    return ret

# An input function for providing input to a model from tf.Examples
def tfexamples_input_fn(examples, feature_spec, label, mode=tf.estimator.ModeKeys.EVAL,
                       num_epochs=None, 
                       batch_size=64):
    def ex_generator():
        for i in range(len(examples)):
            yield examples[i].SerializeToString()
    dataset = tf.data.Dataset.from_generator(
      ex_generator, tf.dtypes.string, tf.TensorShape([]))
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example, label, feature_spec))
    dataset = dataset.repeat(num_epochs)
    return dataset

# Parses Tf.Example protos into features for the input function.
def parse_tf_example(example_proto, label, feature_spec):
    parsed_features = tf.io.parse_example(serialized=example_proto, features=feature_spec)
    target = parsed_features.pop(label)
    return parsed_features, target

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64) or df[col].dtype is np.dtype(np.int32):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        examples.append(example)
    return examples

# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
  df[label_column] = np.where(test(df[label_column]), 1, 0)

#### Read training dataset and stored in df

In [ ]:
import boto3
import sagemaker
import io
s3_bucket = sagemaker.Session().default_bucket()
s3_prefix = 'heidelberg/data'
s3_filename = "transformed.csv"



s3 = boto3.resource('s3')
obj = s3.Object(s3_bucket, s3_prefix+'/'+s3_filename)
body = obj.get()['Body'].read()
df=pd.read_csv(io.BytesIO(body))


#### Specify input columns and column to predict

In [ ]:
features=list(df)
features=features[1:]

In [ ]:
#@title Specify input columns and column to predict {display-mode: "form"}
import numpy as np

# Set the column in the dataset you wish for the model to predict
label_column = 'target'

# Set list of all columns from the dataset we will use for model input.
input_features = features

# Create a list containing all input features and the label column
features_and_labels = input_features + [label_column]

#### Convert dataset to tf.Example protos

In [ ]:
#@title Convert dataset to tf.Example protos {display-mode: "form"}

examples = df_to_examples(df)

#### Create and Train the Regressor

In [ ]:
#@title Create and train the regressor {display-mode: "form"}

num_steps = 200  #@param {type: "number"}

# Create a feature spec for the classifier
feature_spec = create_feature_spec(df, features_and_labels)

# Define and train the classifier
train_inpf = functools.partial(tfexamples_input_fn, examples, feature_spec, label_column)
regressor = tf.estimator.LinearRegressor(
    feature_columns=create_feature_columns(input_features, feature_spec))
regressor.train(train_inpf, steps=num_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpv78wrrdi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initiali

#### Invoke What-If Tool for test data and the trained model

In [ ]:
#@title Invoke What-If Tool for test data and the trained model {display-mode: "form"}

num_datapoints = 2000  #@param {type: "number"}
tool_height_in_px = 1000  #@param {type: "number"}

from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

# Load up the test dataset


# test_csv_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'
# test_df = pd.read_csv(test_csv_path, names=csv_columns, skipinitialspace=True,
#   skiprows=1)
test_examples = df_to_examples(df[0:num_datapoints])

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(test_examples[0:num_datapoints]).set_estimator_and_feature_spec(
    regressor, feature_spec).set_model_type('regression')
WitWidget(config_builder, height=tool_height_in_px)

#### Exploration ideas

- Create a scatter plot of age on the X-axis by inference value on the Y-axis to see how the predicted age compares to the actual age for all test examples.
  - How does the model do overall? Is there a pattern to the errors it creates?

- In the "Performance" tab, set the ground truth feature to "age". You can now see a simple breakdown of the mean error, mean absolute error, and mean squared error over the entire test set. Additionally, you can set a scatter axis or binning option to be any of those three calculations (i.e. you can make a scatter plot of age vs inference error).

- Replace the LinearRegressor with a DNNRegressor (you'll need to decide on the size and number of the hidden layers). Can you create a better model? Does this model have the same pattern of performance issues as the previous model, or are there errors more evenly distributed?
  - If you look at the [WIT Model Comparison](https://colab.research.google.com/github/pair-code/what-if-tool/blob/master/WIT_Model_Comparison.ipynb) notebook, you can see how to compare the two models on a single WIT.

- In the "Performance" tab, slice the dataset by any feature or pair of features. Are there feature values for which the model seems to perform better or worse on?

